In [1]:
##Load metadata from file

import ast
d = {}
nary = {}
i=0
with open("metadata_trial2.txt") as f:
    for line in f:
        
        d = (ast.literal_eval(line))
        nary.update({d['id'] : d})
      # (key, val) = line.split()
      # d[int(key)] = val

In [2]:
#Filter terms that have children and aspect

filnary = {}
kilnary = {}
for key, value in nary.items():
    if value.get('children') != None and value.get('aspect') != None:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect'], 
                    'children' : value['children']})
        kilnary.update({key : filnary})

In [3]:
#import owlready2

import types
from owlready2 import *
onto = get_ontology("http://test.org/onto.owl")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [4]:
#define core classes

with onto:
    class therapeutic_agent(Thing):
        namespace = onto

In [5]:
#classmaker function to make FTC classes

def classmaker(term, parentclass):
                classnameA = 'GOA' + str(term)
                classnameP = 'GOP' + str(term)
                with onto:
                    classname = types.new_class(classnameA, (parentclass,))
                    classname = types.new_class(classnameP, (parentclass,))
                return 0

In [6]:
#Using classmaker to make FTC classes

for key, value in kilnary.items():
    if value['aspect'] == 'biological_process':
        for item in value['children']:
            term = str(value['id'][3:])
            if item['relation'] == 'positively_regulates':
                classmaker(term, therapeutic_agent)
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('biological process', value['id'], 'positively regulates', item['id'])
            if item['relation'] == 'negatively_regulates':
                classmaker(term, therapeutic_agent)
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('biological process', value['id'], 'negatively regulates', item['id'])
            if item['relation'] == 'regulates':
                classmaker(term, therapeutic_agent)
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('biological process', value['id'], 'regulates', item['id'])
                
    if value['aspect'] == 'molecular_function':
        for item in value['children']:
            if item['relation'] == 'positively_regulates':
                classmaker(term, therapeutic_agent)
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('molecular function', value['id'], 'positively regulates', item['id'])
            if item['relation'] == 'negatively_regulates':
                classmaker(term, therapeutic_agent)
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('molecular function', value['id'], 'negatively regulates', item['id'])
            if item['relation'] == 'regulates':
                classmaker(term, therapeutic_agent)
                classmaker(str(item['id'][3:]), therapeutic_agent)
                #print('molecular function', value['id'], 'regulates', item['id'])

In [7]:
#assign appropriate labels to FTC classes

x = list(onto.classes())
for y in x:
    term = 'GO:' + str(y)[8:]
    if str(y)[7] == 'A' and term in kilnary :
        y.label = 'anti_' + kilnary[term]['name'] + '_agent'
    elif str(y)[7] == 'P' and term in kilnary :
        y.label = 'pro_' + kilnary[term]['name'] + '_agent'

In [8]:
#list of ids from the created classes in the two text formats

ids = []
ontoids = []
for cl in list(onto.classes()):
    cl = str(cl)
    ontoids.append(cl)
    clt = cl[5:7] +':' + cl[8:]
    ids.append(clt)

In [9]:
#propertymaker function for making properties

def propertymaker(pname, pdomain, prange):
    with onto:
            classname = types.new_class((pname), (ObjectProperty,), )
            classname = types.new_class((pname), (ObjectProperty,)  )
    
    return 0

In [10]:
#dictionary for domain and range of properties

domdict = {}
randict = {}
for key, value in kilnary.items():
    for item in value['children']:
        if item['id'] in ids:
            propertymaker(item['relation'], value['id'], item['id'])
            domdict[item['relation']] = []
            randict[item['relation']] = []
for key, value in kilnary.items():    
    for item in value['children']:
        if item['id'] in ids and value['id'] in ids:
            domdict[item['relation']].append(ontoids[ids.index(value['id'])])
            randict[item['relation']].append(ontoids[ids.index(item['id'])])

In [11]:
#extracting domains and range as Nonetype objects from respective dictionaries

domdictclist = []
randictclist = []

for key, value in domdict.items():
    for item in value:
        value[value.index(item)] = getattr(onto, item[5:])
        
for key, value in randict.items():
    for item in value:
        value[value.index(item)] = getattr(onto, item[5:])

In [12]:
#test dynamic domain assignment, has acted very strangely

x = list(onto.object_properties())
#for y in x:
    #setattr(y, 'domain', [onto.GOA0008150])

In [13]:
#domain and range assignment to properties

x = list(onto.object_properties())
for y in x:
    ystr = str(y)
    ystr = ystr[5:]
    setattr(y, 'domain', domdict[ystr])
    setattr(y, 'range', randict[ystr])

In [14]:
onto.save(file = "testowl", format = "rdfxml")

In [15]:
list(onto.object_properties())

[onto.is_a,
 onto.positively_regulates,
 onto.negatively_regulates,
 onto.regulates,
 onto.part_of,
 onto.occurs_in]

In [19]:
onto.part_of.domain

[onto.GOA0007131,
 onto.GOA0007131,
 onto.GOA0000070,
 onto.GOA0000070,
 onto.GOA0000070,
 onto.GOA0007052,
 onto.GOA0000079,
 onto.GOA0000278,
 onto.GOA0000278,
 onto.GOA0000278,
 onto.GOA0000278,
 onto.GOA0000278,
 onto.GOA0000278,
 onto.GOA0000278,
 onto.GOA0000278,
 onto.GOA0000278,
 onto.GOA0000278,
 onto.GOA0000278,
 onto.GOA0000288,
 onto.GOA0000747,
 onto.GOA0000749,
 onto.GOA0000819,
 onto.GOA0000910,
 onto.GOA0006383,
 onto.GOA0001302,
 onto.GOA0001503,
 onto.GOA0001525,
 onto.GOA0001541,
 onto.GOA0001547,
 onto.GOA0002040,
 onto.GOA0002040,
 onto.GOA0002062,
 onto.GOA0002063,
 onto.GOA0060041,
 onto.GOA0002181,
 onto.GOA0002181,
 onto.GOA0002181,
 onto.GOA0002183,
 onto.GOA0002353,
 onto.GOA0002367,
 onto.GOA0002367,
 onto.GOA0002377,
 onto.GOA0002455,
 onto.GOA0002456,
 onto.GOA0002510,
 onto.GOA0002510,
 onto.GOA0002514,
 onto.GOA0002526,
 onto.GOA0002544,
 onto.GOA0002775,
 onto.GOA0002775,
 onto.GOA0002778,
 onto.GOA0002778,
 onto.GOA0002781,
 onto.GOA0002781,
 onto.GOA0

In [28]:
method_to_call = getattr(onto, 'GOA0008150')
print(method_to_call())

onto.goa00081502


In [21]:
onto.GOA0008150.part_of = [onto.GOA0007131]

print(onto.GOA0008150.part_of)

[onto.GOA0007131]


In [ ]:
 setattr(onto.GOA0008150, 'is_a', [onto.GOA0006412])

In [ ]:
# import time
time.sleep(70)
print("you're good")
j = 45

In [4]:
print(j)

45
